In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

from common import *

# 1. Load dataset

In [2]:
from dask import delayed, compute
from dask.distributed import Client
from dask.diagnostics import ProgressBar

In [3]:
data = pd.read_csv(join(PATH.input, 'train_data.csv'))
data.head()

,TurbID,Day,Tmstamp,Wspd,Wdir,Etmp,Itmp,Ndir,Pab1,Pab2,Pab3,Prtv,Patv
0,1,1,00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,00:10,6.17,-3.99,30.73,41.80,25.92,1.0,1.0,1.0,-0.25,494.66
2,1,1,00:20,6.27,-2.18,30.60,41.63,20.91,1.0,1.0,1.0,-0.24,509.76
3,1,1,00:30,6.42,-0.73,30.52,41.52,20.91,1.0,1.0,1.0,-0.26,542.53
4,1,1,00:40,6.25,0.89,30.49,41.38,20.91,1.0,1.0,1.0,-0.23,509.36


In [4]:
%%time
data = generate_full_timestamp(data, drop=True)

CPU times: user 5.32 s, sys: 242 ms, total: 5.56 s
Wall time: 3.54 s


In [5]:
data = data.dropna()

In [6]:
in_seq_len = 5*144
out_seq_len = 2*144
shuffle = False
test_size = 0.2

In [7]:
def split_times(times, in_seq_len, out_seq_len):
    inputs, outputs = [], []
    full_times = set(range(1, 28800+1))
    removed_times = full_times - set(times)
    for i in times:  # 28486
        input  = range(i,              i + in_seq_len)
        output = range(i + in_seq_len, i + in_seq_len + out_seq_len)
        if i + in_seq_len + out_seq_len > 28801:
            break
        if len(set(range(i, i + in_seq_len + out_seq_len)) & removed_times) == 0:
            inputs.append(input);  outputs.append(output)
    return inputs, outputs


    
def make_train_val_test_data(data, in_seq_len, out_seq_len, shuffle, test_size):
    from sklearn.model_selection import train_test_split

    train_x, train_y = [], []
    val_x,   val_y   = [], []
    test_x           = []
    
    for i in tqdm(sorted(pd.unique(data['TurbID']))):
        data_tid = data[data['TurbID'] == i]
        times    = data_tid['Time'].sort_values()
        data_tid = data_tid.set_index('Time')
    
        # Make Train, validation set
        inputs, outputs = split_times(times, in_seq_len, out_seq_len)

        try:
            train_in, val_in, train_out, val_out = train_test_split(inputs, outputs, shuffle=shuffle, test_size=test_size)
        except Exception as e:
            print(e);  print(inputs);  print(outputs)
            continue
        
        train_x += [data_tid.loc[times] for times in train_in]
        train_y += [data_tid.loc[times] for times in train_out]
        val_x   += [data_tid.loc[times] for times in val_in]
        val_y   += [data_tid.loc[times] for times in val_out]
        test_x  += [data_tid.iloc[-in_seq_len:]]
        
    train_x, train_y = np.array(train_x), np.array(train_y)
    val_x,   val_y   = np.array(val_x),   np.array(val_y)
    test_x           = np.array(test_x)
    
    print("Train data(X, y)     :", train_x.shape, train_y.shape)
    print("Validation data(X, y):", val_x.shape, val_y.shape)
    print("Test data(X)         :", test_x.shape)
    return train_x, train_y, val_x, val_y, test_x

In [8]:
%%time

# a, b, c, d, e = make_train_val_test_data(data, in_seq_len, out_seq_len, shuffle, test_size)
rsts = make_train_val_test_data(data, in_seq_len, out_seq_len, shuffle, test_size)
# a, b = make_train_val_test_data(data, seq_len, out_len, shuffle, test_size)

100%|██████████| 134/134 [02:44<00:00,  1.22s/it]


Train data(X, y)     : (173016, 720, 11) (173016, 288, 11)
Validation data(X, y): (43319, 720, 11) (43319, 288, 11)
Test data(X)         : (134, 720, 11)
CPU times: user 2min 58s, sys: 7.8 s, total: 3min 6s
Wall time: 3min 5s


In [9]:
train_x, train_y, val_x, val_y, test_x = rsts